In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

: 

In [ ]:
pip install transformers

In [ ]:
import os

In [ ]:
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_name = "google/gemma-3-1b-it"

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
tokenizer("Hello, World!")

In [ ]:
input_conversation = [
    { "role": "user", "content": "Which is the best place to learn GenAI?" },
    { "role": "assistant", "content": "The best place to learn AI is" }
]

In [ ]:
input_detokens = tokenizer.apply_chat_template(
    conversation=input_conversation,
    tokenize=False,
).to(device)
input_detokens

In [ ]:
input_prompt= "Which is the best place to learn GenAI?"
i_tokens= tokenizer(input_prompt, return_tensors="pt")["input_ids"].to(device)
output_tokens = model.generate(i_tokens)
tokenizer.batch_decode(output_tokens)

In [ ]:
output_label="GenAI Cohort By ChaiCode and Piyush Garg"
full_conversation =input_detokens + output_label + tokenizer.eos_token
full_conversation

In [ ]:
input_tokenized= tokenizer(full_conversation, return_tensors="pt",add_special_tokens=False).to(device)["input_ids"]
input_tokenized

In [ ]:
input_ids = input_tokenized[:, :-1].to(device)
target_ids = input_tokenized[:, 1:].to(device)
print("Input ids:", input_ids)
print("Target ids:", target_ids)

In [ ]:
import torch.nn as nn
def calculate_loss(logits, labels):
    loss_fn =nn.CrossEntropyLoss(reduction='none')
    cross_entropy=loss_fn(logits.view(-1, logits.size(-1)), labels.view(-1))
    return cross_entropy

In [ ]:
import torch
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
).to(device)

In [ ]:
from torch.optim import AdamW
model.train()

optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)

for _ in range(10):
    out =model(input_ids=input_ids)
    loss=calculate_loss(out.logits, target_ids).mean()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    print("Loss:", loss.item())

In [ ]:
input_tokens = tokenizer.apply_chat_template(
    conversation=input_conversation,
    tokenize=True,
    return_tensors="pt",
).to(device)
input_tokens

output_tokens = model.generate(input_tokens, max_new_tokens=25)
print(tokenizer.batch_decode(output_tokens, skip_special_tokens=True))